# QEC Code, CNOT, and CCZ Generation

This notebook contains the code for generating the QEC codes and their associated homomorphic CNOT and logical CCZ operators, as described in https://arxiv.org/abs/2510.07269

Includes:
- Toric[1]
- Pentagon
- Bivariate bicycle/tricycle[2]
- Trivariate bicycle/tricycle[2]

For each of these codes, the following data have been saved (as .pkl) in the /data folder.
- Hx and Hz stabilizer generator matrices
- homomorphism matrices (between the 3D codes and one of its component codes[3])
- physical CCZ tensors

Notes: 

[1]The memory and teleportation experiments use the HGP construction of the toric code. Since the physical CCZ construction requires an algebraic code construction, this version of the toric code was used for simulations with CCZ.

[2]Constructing the BB/BT/TB/TT codes requires the SageMath, GAP, and QDistRnd packages.

[3]In the paper, we picked the 2D component code Q_AB, while for the simulations, we used Q_BC. This was an arbitrary choice and does not change the codes beyond a permutation of variable names.

In [ ]:
from sage.all import *
gap(f'SetPackagePath("QDistRnd", "{[PATH_TO_QDistRnd]}/gap-4.14.0/pkg/qdistrnd");')

In [ ]:
import numpy as np
import joblib
import sys
sys.path.append('../src/')
from CodeGeneration import (create_2D_HGP_code, 
    create_3D_HGP_code, 
    create_homomorphism_from_2D_to_3D_HGP, 
    create_homomorphism_from_2D_to_3D_LP_A, 
    check_homomorphism_commutes
)
from BP_codes_sage import (regular_rep_matrix, 
    lift_matrix_over_group_algebra, 
    BalancedProductCode, 
    DistanceEst_Gap, 
    ga_transpose_reverse
)
gap('LoadPackage("QDistRnd");')

true

## Code construction + homomorphisms

In [ ]:
hx_dict_2D, hz_dict_2D = {}, {}
hx_dict_3D, hz_dict_3D = {}, {}
gamma_0_dict, gamma_1_dict, gamma_2_dict, gamma_3_dict = {}, {}, {}, {}

### Toric Codes

In [ ]:
H_ring_d3 = np.array([[1, 1, 0],
                      [0, 1, 1],
                      [1, 0, 1]])

code_2D = create_2D_HGP_code(H_ring_d3, H_ring_d3.T, compute_distance=False)
code_2D.D = min(DistanceEst_Gap(code_2D))
code_3D = create_3D_HGP_code(H_ring_d3.T, H_ring_d3, H_ring_d3.T, compute_distance=False)
code_3D.D = min(DistanceEst_Gap(code_3D))
print("2D toric code:", code_2D.code_params)
print("3D toric code:", code_3D.code_params)

gamma_0, gamma_1, gamma_2, gamma_3 = create_homomorphism_from_2D_to_3D_HGP(H_ring_d3.T, H_ring_d3, H_ring_d3.T, 0)
print("check gamma_0 and X checks commute", check_homomorphism_commutes(code_3D.hx, code_2D.hx, gamma_1, gamma_0))
print("check gamma_2 and Z checks commute", check_homomorphism_commutes(code_3D.hz.T, code_2D.hz.T, gamma_2, gamma_1))

2D toric code: (nan,nan)-[[18,2,3]]
3D toric code: (nan,nan)-[[81,3,3]]
check gamma_0 and X checks commute True
check gamma_2 and Z checks commute True


In [ ]:
code_type = "toric"
d = 3
code_id = f"d{d}_{code_type}"
hx_dict_2D[code_id], hz_dict_2D[code_id] = code_2D.hx.toarray(), code_2D.hz.toarray()
hx_dict_3D[code_id], hz_dict_3D[code_id] = code_3D.hx.toarray(), code_3D.hz.toarray()
gamma_0_dict[code_id], gamma_1_dict[code_id], gamma_2_dict[code_id], gamma_3_dict[code_id] = gamma_0, gamma_1, gamma_2, gamma_3

In [ ]:
H_ring_d5 = np.array([[1, 1, 0, 0, 0],
                      [0, 1, 1, 0, 0],
                      [0, 0, 1, 1, 0],
                      [0, 0, 0, 1, 1],
                      [1, 0, 0, 0, 1]])

code_2D = create_2D_HGP_code(H_ring_d5, H_ring_d5.T, compute_distance=False)
code_2D.D = min(DistanceEst_Gap(code_2D))
code_3D = create_3D_HGP_code(H_ring_d5.T, H_ring_d5, H_ring_d5.T, compute_distance=False)
code_3D.D = min(DistanceEst_Gap(code_3D))
print("2D toric code:", code_2D.code_params)
print("3D toric code:", code_3D.code_params)

gamma_0, gamma_1, gamma_2, gamma_3 = create_homomorphism_from_2D_to_3D_HGP(H_ring_d5.T, H_ring_d5, H_ring_d5.T, 0)
print("check gamma_0 and X checks commute", check_homomorphism_commutes(code_3D.hx, code_2D.hx, gamma_1, gamma_0))
print("check gamma_2 and Z checks commute", check_homomorphism_commutes(code_3D.hz.T, code_2D.hz.T, gamma_2, gamma_1))

2D toric code: (nan,nan)-[[50,2,5]]
3D toric code: (nan,nan)-[[375,3,5]]
check gamma_0 and X checks commute True
check gamma_2 and Z checks commute True


In [ ]:
code_type = "toric"
d = 5
code_id = f"d{d}_{code_type}"
hx_dict_2D[code_id], hz_dict_2D[code_id] = code_2D.hx.toarray(), code_2D.hz.toarray()
hx_dict_3D[code_id], hz_dict_3D[code_id] = code_3D.hx.toarray(), code_3D.hz.toarray()
gamma_0_dict[code_id], gamma_1_dict[code_id], gamma_2_dict[code_id], gamma_3_dict[code_id] = gamma_0, gamma_1, gamma_2, gamma_3

### Pentagon Codes

In [ ]:
H_pentagon = np.array([[1., 0., 0., 0., 1., 1., 0., 0., 1., 0.],
       [1., 1., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 1., 1., 0., 0., 1., 0., 1., 0., 0.],
       [0., 0., 1., 1., 0., 0., 1., 0., 1., 0.],
       [0., 0., 0., 1., 1., 0., 0., 1., 0., 1.]])

# combined with H_ring_d3
H_ring_d3 = np.array([[1, 1, 0],
                      [0, 1, 1],
                      [1, 0, 1]])

code_3D = create_3D_HGP_code(H_ring_d3.T, H_ring_d3.T, H_pentagon, compute_distance=False)
code_3D.D = min(DistanceEst_Gap(code_3D))
code_2D = create_2D_HGP_code(H_ring_d3.T, H_pentagon, compute_distance=False)
code_2D.D = min(DistanceEst_Gap(code_2D))
print("2D pentagon code:", code_2D.code_params)
print("3D pentagon code:", code_3D.code_params)

gamma_0, gamma_1, gamma_2, gamma_3 = create_homomorphism_from_2D_to_3D_HGP(H_ring_d3.T, H_ring_d3.T, H_pentagon, 0)
print("check gamma_0 and X checks commute", check_homomorphism_commutes(code_3D.hx, code_2D.hx, gamma_1, gamma_0))
print("check gamma_2 and Z checks commute", check_homomorphism_commutes(code_3D.hz.T, code_2D.hz.T, gamma_2, gamma_1))

2D pentagon code: (nan,nan)-[[45,7,3]]
3D pentagon code: (nan,nan)-[[180,8,3]]
check gamma_0 and X checks commute True
check gamma_2 and Z checks commute True


In [ ]:
code_type = "pentagon"
d = 3
code_id = f"d{d}_{code_type}"
hx_dict_2D[code_id], hz_dict_2D[code_id] = code_2D.hx.toarray(), code_2D.hz.toarray()
hx_dict_3D[code_id], hz_dict_3D[code_id] = code_3D.hx.toarray(), code_3D.hz.toarray()
gamma_0_dict[code_id], gamma_1_dict[code_id], gamma_2_dict[code_id], gamma_3_dict[code_id] = gamma_0, gamma_1, gamma_2, gamma_3

### Bivariate bicycle/tricycle codes

sagevenv/bin/python

In [ ]:
# [[27, 3, 3]]
lx, ly = 3, 3

G = AbelianGroup([lx, ly], names=("x", "y"))
G_basis = G.list()
one = G.identity()
x, y = G.gens()
R = GroupAlgebra(G, GF(2))

binomials = {'A': R(one) + R(x*y), 'B': R(one) + R(y), 'C': R(one) + R(x*y**2)}
H1 = Matrix(R, [[binomials['A']]])
H2 = Matrix(R, [[binomials['B']]])
H3 = Matrix(R, [[binomials['C']]])

code_3D_obj = BalancedProductCode(H1, H2, H3)
code_3D = code_3D_obj.code
code_3D.D = min(DistanceEst_Gap(code_3D))
print(code_3D.code_params)

# puncture A
code_2D_obj = BalancedProductCode(H2, H3)
code_2D = code_2D_obj.code
code_2D.D = min(DistanceEst_Gap(code_2D))
print(code_2D.code_params)

gamma_0, gamma_1, gamma_2, gamma_3 = create_homomorphism_from_2D_to_3D_LP_A(H1, H2, H3, R, G, one, 0)
print("check gamma_0 and X checks commute", check_homomorphism_commutes(code_3D.hx, code_2D.hx, gamma_1, gamma_0))
print("check gamma_2 and Z checks commute", check_homomorphism_commutes(code_3D.hz.T, code_2D.hz.T, gamma_2, gamma_1))

(nan,nan)-[[27,3,3]]
(nan,nan)-[[18,2,3]]
check gamma_0 and X checks commute True
check gamma_2 and Z checks commute True


In [ ]:
code_type = "blp"
d = 3
code_id = f"d{d}_{code_type}"
hx_dict_2D[code_id], hz_dict_2D[code_id] = code_2D.hx.toarray(), code_2D.hz.toarray()
hx_dict_3D[code_id], hz_dict_3D[code_id] = code_3D.hx.toarray(), code_3D.hz.toarray()
gamma_0_dict[code_id], gamma_1_dict[code_id], gamma_2_dict[code_id], gamma_3_dict[code_id] = gamma_0, gamma_1, gamma_2, gamma_3

In [ ]:
# [[81, 3, 5]]
lx, ly = 3, 9

G = AbelianGroup([lx, ly], names=("x", "y"))
G_basis = G.list()
one = G.identity()
x, y = G.gens()
R = GroupAlgebra(G, GF(2))

binomials = {'A': R(x**2*y**4) + R(x**2*y**6), 'B': R(x*y**3) + R(x**2*y), 'C': R(one) + R(x*y**8)}
H1 = Matrix(R, [[binomials['A']]])
H2 = Matrix(R, [[binomials['B']]])
H3 = Matrix(R, [[binomials['C']]])

code_3D_obj = BalancedProductCode(H1, H2, H3)
code_3D = code_3D_obj.code
code_3D.D = min(DistanceEst_Gap(code_3D))
print(code_3D.code_params)

# puncture A
code_2D_obj = BalancedProductCode(H2, H3)
code_2D = code_2D_obj.code
code_2D.D = min(DistanceEst_Gap(code_2D))
print(code_2D.code_params)

gamma_0, gamma_1, gamma_2, gamma_3 = create_homomorphism_from_2D_to_3D_LP_A(H1, H2, H3, R, G, one, 0)
print("check gamma_0 and X checks commute", check_homomorphism_commutes(code_3D.hx, code_2D.hx, gamma_1, gamma_0))
print("check gamma_2 and Z checks commute", check_homomorphism_commutes(code_3D.hz.T, code_2D.hz.T, gamma_2, gamma_1))

(nan,nan)-[[81,3,5]]
(nan,nan)-[[54,2,6]]
check gamma_0 and X checks commute True
check gamma_2 and Z checks commute True


In [ ]:
code_type = "blp"
d = 5
code_id = f"d{d}_{code_type}"
hx_dict_2D[code_id], hz_dict_2D[code_id] = code_2D.hx.toarray(), code_2D.hz.toarray()
hx_dict_3D[code_id], hz_dict_3D[code_id] = code_3D.hx.toarray(), code_3D.hz.toarray()
gamma_0_dict[code_id], gamma_1_dict[code_id], gamma_2_dict[code_id], gamma_3_dict[code_id] = gamma_0, gamma_1, gamma_2, gamma_3

### Trivariate bicycle/tricycle Codes

In [ ]:
lx, ly, lz = 2, 5, 7

G = AbelianGroup([lx, ly, lz], names=("x", "y", "z"))
G_basis = G.list()
one = G.identity()
x, y, z = G.gens()
R = GroupAlgebra(G, GF(2))

binomials = {'A': R(y*z) + R(y**4*z**2), 'B': R(y**2*z**2) + R( x*y**2*z**4), 'C': R(one) + R(x*y**2*z**3)}
print(binomials)
H1 = Matrix(R, [[binomials['A']]])
H2 = Matrix(R, [[binomials['B']]])
H3 = Matrix(R, [[binomials['C']]])

code_3D_obj = BalancedProductCode(H1, H2, H3)
code_3D = code_3D_obj.code
code_3D.D = min(DistanceEst_Gap(code_3D))
print(code_3D.code_params)

code_2D_obj = BalancedProductCode(H2, H3)
code_2D = code_2D_obj.code
code_2D.D = min(DistanceEst_Gap(code_2D))
print(code_2D.code_params)

gamma_0, gamma_1, gamma_2, gamma_3 = create_homomorphism_from_2D_to_3D_LP_A(H1, H2, H3, R, G, one, 0)
print("check gamma_0 and X checks commute", check_homomorphism_commutes(code_3D.hx, code_2D.hx, gamma_1, gamma_0))
print("check gamma_2 and Z checks commute", check_homomorphism_commutes(code_3D.hz.T, code_2D.hz.T, gamma_2, gamma_1))

{'A': y*z + y^4*z^2, 'B': y^2*z^2 + x*y^2*z^4, 'C': 1 + x*y^2*z^3}
(nan,nan)-[[210,3,7]]
(nan,nan)-[[140,2,8]]
check gamma_0 and X checks commute True
check gamma_2 and Z checks commute True


In [ ]:
code_type = "tlp"
d = 7
code_id = f"d{d}_{code_type}"
hx_dict_2D[code_id], hz_dict_2D[code_id] = code_2D.hx.toarray(), code_2D.hz.toarray()
hx_dict_3D[code_id], hz_dict_3D[code_id] = code_3D.hx.toarray(), code_3D.hz.toarray()
gamma_0_dict[code_id], gamma_1_dict[code_id], gamma_2_dict[code_id], gamma_3_dict[code_id] = gamma_0, gamma_1, gamma_2, gamma_3

### algebraic construction of toric codes

In [ ]:
lx, ly, lz = 3, 3, 3

G = AbelianGroup([lx, ly, lz], names=("x", "y", "z"))
G_basis = G.list()
one = G.identity()
x, y, z = G.gens()
R = GroupAlgebra(G, GF(2))

binomials = {'A': R(one) + R(x), 'B': R(one) + R(y), 'C': R(one) + R(z)}
print(binomials)
H1 = Matrix(R, [[binomials['A']]])
H2 = Matrix(R, [[binomials['B']]])
H3 = Matrix(R, [[binomials['C']]])

code_3D_obj = BalancedProductCode(H1, H2, H3)
code_3D = code_3D_obj.code
code_3D.D = min(DistanceEst_Gap(code_3D, trials = 30))
print(code_3D.code_params)

{'A': 1 + x, 'B': 1 + y, 'C': 1 + z}
(nan,nan)-[[81,3,3]]


In [ ]:
code_type = "algebraic_toric"
d = 3
code_id = f"d{d}_{code_type}"
hx_dict_3D[code_id], hz_dict_3D[code_id] = code_3D.hx.toarray(), code_3D.hz.toarray()

In [ ]:
lx, ly, lz = 5, 5, 5

G = AbelianGroup([lx, ly, lz], names=("x", "y", "z"))
G_basis = G.list()
one = G.identity()
x, y, z = G.gens()
R = GroupAlgebra(G, GF(2))

binomials = {'A': R(one) + R(x), 'B': R(one) + R(y), 'C': R(one) + R(z)}
print(binomials)
H1 = Matrix(R, [[binomials['A']]])
H2 = Matrix(R, [[binomials['B']]])
H3 = Matrix(R, [[binomials['C']]])

code_3D_obj = BalancedProductCode(H1, H2, H3)
code_3D = code_3D_obj.code
code_3D.D = min(DistanceEst_Gap(code_3D, trials = 30))
print(code_3D.code_params)

{'A': 1 + x, 'B': 1 + y, 'C': 1 + z}
(nan,nan)-[[375,3,5]]


In [ ]:
code_type = "algebraic_toric"
d = 5
code_id = f"d{d}_{code_type}"
hx_dict_3D[code_id], hz_dict_3D[code_id] = code_3D.hx.toarray(), code_3D.hz.toarray()

### save codes to .pkl file

In [ ]:
# verify all dicts contain the correct codes

from bposd.css import css_code

for code_id in hx_dict_3D.keys():
    print(code_id)
    if code_id in hx_dict_2D:
        hx = hx_dict_2D[code_id]
        hz = hz_dict_2D[code_id]
        code_2D = css_code(hx, hz)
        code_2D.D = min(DistanceEst_Gap(code_2D))
        print("2D", code_2D.code_params)
    hx = hx_dict_3D[code_id]
    hz = hz_dict_3D[code_id]
    code_3D = css_code(hx, hz)
    code_3D.D = min(DistanceEst_Gap(code_3D))
    print("3D", code_3D.code_params)

d3_toric
2D (nan,nan)-[[18,2,3]]
3D (nan,nan)-[[81,3,3]]
d5_toric
2D (nan,nan)-[[50,2,5]]
3D (nan,nan)-[[375,3,5]]
d3_pentagon
2D (nan,nan)-[[45,7,3]]
3D (nan,nan)-[[180,8,3]]
d3_blp
2D (nan,nan)-[[18,2,3]]
3D (nan,nan)-[[27,3,3]]
d5_blp
2D (nan,nan)-[[54,2,6]]
3D (nan,nan)-[[81,3,5]]
d7_tlp
2D (nan,nan)-[[140,2,8]]
3D (nan,nan)-[[210,3,7]]
d3_algebraic_toric
3D (nan,nan)-[[81,3,3]]
d5_algebraic_toric
3D (nan,nan)-[[375,3,5]]


In [ ]:
joblib.dump(hx_dict_2D, '../data/hx_dict_2D.pkl')
joblib.dump(hz_dict_2D, '../data/hz_dict_2D.pkl')
joblib.dump(hx_dict_3D, '../data/hx_dict_3D.pkl')
joblib.dump(hz_dict_3D, '../data/hz_dict_3D.pkl')
joblib.dump(gamma_0_dict, '../data/gamma_0_dict.pkl')
joblib.dump(gamma_1_dict, '../data/gamma_1_dict.pkl')
joblib.dump(gamma_2_dict, '../data/gamma_2_dict.pkl')
joblib.dump(gamma_3_dict, '../data/gamma_3_dict.pkl')


['../data/gamma_3_dict.pkl']

## Physical CCZ tensor construction

In [ ]:
from PCCZMatrix import generate_triples, generate_PCCZ_bivariate, generate_PCCZ_trivariate
from BP_cup_prod import VerifyCohomology_CZ, VerifyCohomology_CCZ

In [ ]:
P_CCZ_dict = {}

### [[27, 3, 3]] bivariate tricycle

In [ ]:
lx, ly = 3, 3

G = AbelianGroup([lx, ly], names=("x", "y"))
G_basis = G.list()
one = G.identity()
x, y = G.gens()
R = GroupAlgebra(G, GF(2))

binomials = {'A': R(x) + R(x**2*y), 'B': R(x**2*y) + R(x**2*y**2), 'C': R(one) + R(x*y**2)}
print(binomials)
H1 = Matrix(R, [[binomials['A']]])
H2 = Matrix(R, [[binomials['B']]])
H3 = Matrix(R, [[binomials['C']]])

code_3D_obj = BalancedProductCode(H1, H2, H3)
code_3D = code_3D_obj.code
code_3D.D = min(DistanceEst_Gap(code_3D))
print(code_3D.code_params)

n0 = int(code_3D.N/3)
n = code_3D.N

{'A': x + x^2*y, 'B': x^2*y + x^2*y^2, 'C': 1 + x*y^2}
(nan,nan)-[[27,3,3]]


In [ ]:
# specify the orientations for the classical codes
# a
Ha_in = Matrix(R, [[R(x)]])
Ha_out = Matrix(R, [[R(x**2*y)]])
Ha_free = Matrix(R, [[R(0)]])

delta_in, delta_out, delta_free = ga_transpose_reverse(Ha_in), ga_transpose_reverse(Ha_out), ga_transpose_reverse(Ha_free)
delta_a = delta_in + delta_out + delta_free
orientation_mats_a = {'delta_in': delta_in,  'delta_out': delta_out, 'delta_free': delta_free}

# b
Ha_in = Matrix(R, [[R(x**2*y)]])
Ha_out = Matrix(R, [[R(x**2*y**2)]])
Ha_free = Matrix(R, [[R(0)]])

delta_in, delta_out, delta_free = ga_transpose_reverse(Ha_in), ga_transpose_reverse(Ha_out), ga_transpose_reverse(Ha_free)
delta_b = delta_in + delta_out + delta_free
orientation_mats_b = {'delta_in': delta_in,  'delta_out': delta_out, 'delta_free': delta_free}

# c
Ha_in = Matrix(R, [[R(one)]])
Ha_out = Matrix(R, [[R(x*y**2)]])
Ha_free = Matrix(R, [[R(0)]])

delta_in, delta_out, delta_free = ga_transpose_reverse(Ha_in), ga_transpose_reverse(Ha_out), ga_transpose_reverse(Ha_free)
delta_c = delta_in + delta_out + delta_free
orientation_mats_c = {'delta_in': delta_in,  'delta_out': delta_out, 'delta_free': delta_free}

deltas = {'delta_a':delta_a, 'delta_b':delta_b, 'delta_c':delta_c}
orientation_mats = {'orientation_mats_a':orientation_mats_a, 'orientation_mats_b':orientation_mats_b, 'orientation_mats_c':orientation_mats_c}

In [ ]:
def MonomialToQIx(alpha, b_ix):
    mat = regular_rep_matrix(alpha, G_basis)
    m_ix = np.where(np.array(mat[:, 0]) == 1)[0][0]
    return b_ix*n0 + m_ix

triples = generate_triples(orientation_mats_a, orientation_mats_b, orientation_mats_c)
P_CCZ_mat = generate_PCCZ_bivariate(n, R, x, y, lx, ly, triples, MonomialToQIx)
VerifyCohomology_CCZ(code_3D, P_CCZ_mat)

True

In [ ]:
code_type = "blp"
d = 3
code_id = f"d{d}_{code_type}"
P_CCZ_dict[code_id] = P_CCZ_mat

### [[81, 3, 5]] bivariate tricycle

In [ ]:
lx, ly = 3, 9

G = AbelianGroup([lx, ly], names=("x", "y"))
G_basis = G.list()
one = G.identity()
x, y = G.gens()
R = GroupAlgebra(G, GF(2))

binomials = {'A': R(x**2*y**4) + R(x**2*y**6), 'B': R(x*y**3) + R(x**2*y), 'C': R(one) + R(x*y**8)}
H1 = Matrix(R, [[binomials['A']]])
H2 = Matrix(R, [[binomials['B']]])
H3 = Matrix(R, [[binomials['C']]])

code_3D_obj = BalancedProductCode(H1, H2, H3)
code_3D = code_3D_obj.code
code_3D.D = min(DistanceEst_Gap(code_3D))
print(code_3D.code_params)

n0 = int(code_3D.N/3)
n = code_3D.N

(nan,nan)-[[81,3,5]]


In [ ]:
# specify the orientations for the classical codes
# a
Ha_in = Matrix(R, [[R(x**2*y**4)]])
Ha_out = Matrix(R, [[R(x**2*y**6)]])
Ha_free = Matrix(R, [[R(0)]])

delta_in, delta_out, delta_free = ga_transpose_reverse(Ha_in), ga_transpose_reverse(Ha_out), ga_transpose_reverse(Ha_free)
delta_a = delta_in + delta_out + delta_free
orientation_mats_a = {'delta_in': delta_in,  'delta_out': delta_out, 'delta_free': delta_free}

# b
Ha_in = Matrix(R, [[R(x*y**3)]])
Ha_out = Matrix(R, [[R(x**2*y)]])
Ha_free = Matrix(R, [[R(0)]])

delta_in, delta_out, delta_free = ga_transpose_reverse(Ha_in), ga_transpose_reverse(Ha_out), ga_transpose_reverse(Ha_free)
delta_b = delta_in + delta_out + delta_free
orientation_mats_b = {'delta_in': delta_in,  'delta_out': delta_out, 'delta_free': delta_free}

# c
Ha_in = Matrix(R, [[R(one)]])
Ha_out = Matrix(R, [[R(x*y**8)]])
Ha_free = Matrix(R, [[R(0)]])

delta_in, delta_out, delta_free = ga_transpose_reverse(Ha_in), ga_transpose_reverse(Ha_out), ga_transpose_reverse(Ha_free)
delta_c = delta_in + delta_out + delta_free
orientation_mats_c = {'delta_in': delta_in,  'delta_out': delta_out, 'delta_free': delta_free}

# combining
deltas = {'delta_a':delta_a, 'delta_b':delta_b, 'delta_c':delta_c}
orientation_mats = {'orientation_mats_a':orientation_mats_a, 'orientation_mats_b':orientation_mats_b, 'orientation_mats_c':orientation_mats_c}

In [ ]:
def MonomialToQIx(alpha, b_ix):
    mat = regular_rep_matrix(alpha, G_basis)
    m_ix = np.where(np.array(mat[:, 0]) == 1)[0][0]
    return b_ix*n0 + m_ix

# get physical CCZ matrix and verify its cohomology is correct
triples = generate_triples(orientation_mats_a, orientation_mats_b, orientation_mats_c)
P_CCZ_mat = generate_PCCZ_bivariate(n, R, x, y, lx, ly, triples, MonomialToQIx)
VerifyCohomology_CCZ(code_3D, P_CCZ_mat)

True

In [ ]:
code_type = "blp"
d = 5
code_id = f"d{d}_{code_type}"
P_CCZ_dict[code_id] = P_CCZ_mat

### [[81, 3, 3]] 3D toric

In [ ]:
lx, ly, lz = 3, 3, 3

G = AbelianGroup([lx, ly, lz], names=("x", "y", "z"))
G_basis = G.list()
one = G.identity()
x, y, z = G.gens()
R = GroupAlgebra(G, GF(2))

binomials = {'A': R(one) + R(x), 'B': R(one) + R(y), 'C': R(one) + R(z)}
print(binomials)
H1 = Matrix(R, [[binomials['A']]])
H2 = Matrix(R, [[binomials['B']]])
H3 = Matrix(R, [[binomials['C']]])

code_3D_obj = BalancedProductCode(H1, H2, H3)
code_3D = code_3D_obj.code
code_3D.D = min(DistanceEst_Gap(code_3D, trials = 30))
print(code_3D.code_params)

n0 = int(code_3D.N/3)
n = code_3D.N

{'A': 1 + x, 'B': 1 + y, 'C': 1 + z}
(nan,nan)-[[81,3,3]]


In [ ]:
# get map from (alpha, ix) to qubit_ix
def MonomialToQIx(alpha, b_ix):
    mat = regular_rep_matrix(alpha, G_basis)
    m_ix = np.where(np.array(mat[:, 0]) == 1)[0][0]
    return b_ix*n0 + m_ix

# specify the orientations for the classical codes
# a
Ha_in = Matrix(R, [[R(one)]])
Ha_out = Matrix(R, [[R(x)]])
Ha_free = Matrix(R, [[R(0)]])

delta_in, delta_out, delta_free = ga_transpose_reverse(Ha_in), ga_transpose_reverse(Ha_out), ga_transpose_reverse(Ha_free)
delta_a = delta_in + delta_out + delta_free
orientation_mats_a = {'delta_in': delta_in,  'delta_out': delta_out, 'delta_free': delta_free}

# b
Ha_in = Matrix(R, [[R(one)]])
Ha_out = Matrix(R, [[R(y)]])
Ha_free = Matrix(R, [[R(0)]])

delta_in, delta_out, delta_free = ga_transpose_reverse(Ha_in), ga_transpose_reverse(Ha_out), ga_transpose_reverse(Ha_free)
delta_b = delta_in + delta_out + delta_free
orientation_mats_b = {'delta_in': delta_in,  'delta_out': delta_out, 'delta_free': delta_free}

# c
Ha_in = Matrix(R, [[R(one)]])
Ha_out = Matrix(R, [[R(z)]])
Ha_free = Matrix(R, [[R(0)]])

delta_in, delta_out, delta_free = ga_transpose_reverse(Ha_in), ga_transpose_reverse(Ha_out), ga_transpose_reverse(Ha_free)
delta_c = delta_in + delta_out + delta_free
orientation_mats_c = {'delta_in': delta_in,  'delta_out': delta_out, 'delta_free': delta_free}

deltas = {'delta_a':delta_a, 'delta_b':delta_b, 'delta_c':delta_c}
orientation_mats = {'orientation_mats_a':orientation_mats_a, 'orientation_mats_b':orientation_mats_b, 'orientation_mats_c':orientation_mats_c}

In [ ]:
def MonomialToQIx(alpha, b_ix):
    mat = regular_rep_matrix(alpha, G_basis)
    m_ix = np.where(np.array(mat[:, 0]) == 1)[0][0]
    return b_ix*n0 + m_ix

# get physical CCZ matrix and verify its cohomology is correct
triples = generate_triples(orientation_mats_a, orientation_mats_b, orientation_mats_c)
P_CCZ_mat = generate_PCCZ_trivariate(n, R, x, y, z, lx, ly, lz, triples, MonomialToQIx)
# VerifyCohomology_CCZ(code_3D, P_CCZ_mat)

In [ ]:
code_type = "algebraic_toric"
d = 3
code_id = f"d{d}_{code_type}"
P_CCZ_dict[code_id] = P_CCZ_mat

### [[375, 3, 5]] 3D toric

In [ ]:
lx, ly, lz = 5, 5, 5

G = AbelianGroup([lx, ly, lz], names=("x", "y", "z"))
G_basis = G.list()
one = G.identity()
x, y, z = G.gens()
R = GroupAlgebra(G, GF(2))

binomials = {'A': R(one) + R(x), 'B': R(one) + R(y), 'C': R(one) + R(z)}
print(binomials)
H1 = Matrix(R, [[binomials['A']]])
H2 = Matrix(R, [[binomials['B']]])
H3 = Matrix(R, [[binomials['C']]])

code_3D_obj = BalancedProductCode(H1, H2, H3)
code_3D = code_3D_obj.code
code_3D.D = min(DistanceEst_Gap(code_3D, trials = 30))
print(code_3D.code_params)

n0 = int(code_3D.N/3)
n = code_3D.N

{'A': 1 + x, 'B': 1 + y, 'C': 1 + z}
(nan,nan)-[[375,3,5]]


In [ ]:
# specify the orientations for the classical codes
# a
Ha_in = Matrix(R, [[R(one)]])
Ha_out = Matrix(R, [[R(x)]])
Ha_free = Matrix(R, [[R(0)]])

delta_in, delta_out, delta_free = ga_transpose_reverse(Ha_in), ga_transpose_reverse(Ha_out), ga_transpose_reverse(Ha_free)
delta_a = delta_in + delta_out + delta_free
orientation_mats_a = {'delta_in': delta_in,  'delta_out': delta_out, 'delta_free': delta_free}

# b
Ha_in = Matrix(R, [[R(one)]])
Ha_out = Matrix(R, [[R(y)]])
Ha_free = Matrix(R, [[R(0)]])

delta_in, delta_out, delta_free = ga_transpose_reverse(Ha_in), ga_transpose_reverse(Ha_out), ga_transpose_reverse(Ha_free)
delta_b = delta_in + delta_out + delta_free
orientation_mats_b = {'delta_in': delta_in,  'delta_out': delta_out, 'delta_free': delta_free}

# c
Ha_in = Matrix(R, [[R(one)]])
Ha_out = Matrix(R, [[R(z)]])
Ha_free = Matrix(R, [[R(0)]])

delta_in, delta_out, delta_free = ga_transpose_reverse(Ha_in), ga_transpose_reverse(Ha_out), ga_transpose_reverse(Ha_free)
delta_c = delta_in + delta_out + delta_free
orientation_mats_c = {'delta_in': delta_in,  'delta_out': delta_out, 'delta_free': delta_free}

deltas = {'delta_a':delta_a, 'delta_b':delta_b, 'delta_c':delta_c}
orientation_mats = {'orientation_mats_a':orientation_mats_a, 'orientation_mats_b':orientation_mats_b, 'orientation_mats_c':orientation_mats_c}

In [ ]:
def MonomialToQIx(alpha, b_ix):
    mat = regular_rep_matrix(alpha, G_basis)
    m_ix = np.where(np.array(mat[:, 0]) == 1)[0][0]
    return b_ix*n0 + m_ix

# get physical CCZ matrix and verify its cohomology is correct
triples = generate_triples(orientation_mats_a, orientation_mats_b, orientation_mats_c)
P_CCZ_mat = generate_PCCZ_trivariate(n, R, x, y, z, lx, ly, lz, triples, MonomialToQIx)
# VerifyCohomology_CCZ(code_3D, P_CCZ_mat)

In [ ]:
# code_type = "algebraic_toric"
# d = 5
# code_id = f"d{d}_{code_type}"
# P_CCZ_dict[code_id] = P_CCZ_mat

### save P_CCZ dict to .pkl file

Note: this is not uploaded to the Github repo because file size is too large.

In [ ]:
joblib.dump(P_CCZ_dict, '../data/P_CCZ_dict.pkl')

['../data/P_CCZ_dict.pkl']